# Extracting symbols from image using `OpenCV`

In [254]:
import os
import pickle
import cv2
import numpy as np
from functools import cmp_to_key
%matplotlib inline
from IPython.display import display, Image

In [249]:
def extractSymbols(imgOrig, showSteps = False):
    debugImgSteps = []
    imgGray = cv2.cvtColor(imgOrig,cv2.COLOR_BGR2GRAY)
    imgFiltered = cv2.medianBlur(imgGray, 5)
    debugImgSteps.append(imgFiltered)
    
    imgCanny = cv2.Canny(imgFiltered, 50,180)
    debugImgSteps.append(imgCanny)

    kernel = np.ones((5,5), np.uint8)
    imgDilated = cv2.dilate(imgCanny, kernel, iterations=5)
    debugImgSteps.append(imgDilated)

    contours, _= cv2.findContours(imgDilated, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

    boundingBoxes = []
    for contour in contours:
        x,y,w,h = cv2.boundingRect(contour)
        boundingBoxes.append((x,y,w,h))

    global rowsG
    rowsG, _, _ = imgOrig.shape
    key_leftRightTopBottom = cmp_to_key(leftRightTopBottom)
    boundingBoxes = sorted(boundingBoxes, key=key_leftRightTopBottom)

    symbols = []
    for (i, box) in enumerate(boundingBoxes):
        x,y,w,h = box
        mathSymbol = imgOrig[y:y+h, x:x+w]
        mathSymbol = cv2.cvtColor(mathSymbol, cv2.COLOR_BGR2GRAY) #converting to Gray as tensorflow deals with grayscale or RGB, not BGR
        mathSymbol = cv2.resize(mathSymbol, (45,45), interpolation=cv2.INTER_AREA) #to have the same size as trained images in the dataset
        debugImgSteps.append(mathSymbol)
        mathSymbolF = mathSymbol.astype('float32') #optional: tensorflows deals with float32, not uint8
        symbols.append(mathSymbolF)

    if showSteps:
        dispImages(debugImgSteps)

    return symbols
        

In [250]:
def leftRightTopBottom(tup1, tup2):
    x1, y1, _, _ = tup1
    x2, y2, _, _ = tup2
    rows = rowsG
    yRegion1, yRegion2 = -1, -1

    for i in range(8):
        if y1 < rows/8 + rows*(i/8):
            yRegion1 = i
            break
    else:
        if yRegion1 == -1:
            yRegion1 = 8

    for i in range(8):
        if y2 < rows/8 + rows*(i/8):
            yRegion2 = i
            break
    else:
        if yRegion2 == -1:
            yRegion2 = 8
    
    if yRegion1 < yRegion2:
        return -1
    elif yRegion2 < yRegion1:
        return 1
    elif x1 <= x2:
        return -1
    else:
        return 1


In [251]:
def dispImages(imgs):
    for img in imgs:
        cv2.imshow('Image', img)
        cv2.waitKey(0)
    else:
        cv2.destroyAllWindows()

In [252]:
img = cv2.imread('tests/testMath5.png')
symbols = extractSymbols(img, showSteps=True)


# Creating dictionary that maps folder names to latex

example of folder names in "mathSymbolsDataset": <br>
<img src="guideImages/datasetFolders.png" width=400 height=300>

using `r` to make the string `raw` to avoid confusing strings like `\n` with python's new line <br>
however, the values will now have two backslashes (e.g. `\\n`), thus, we will later need to replace each `\\` with `\`

In [343]:
dic = {
    "-": r"-",
    "!": r"!",
    "(": r"(",
    ")": r")",
    ",": r",",
    "[": r"[",
    "]": r"]",
    "{": r"\{",
    "}": r"\}",
    "+": r"+",
    "=": r"=",
    "0": r"0",
    "1": r"1",
    "2": r"2",
    "3": r"3",
    "4": r"4",
    "5": r"5",
    "6": r"6",
    "7": r"7",
    "8": r"8",
    "9": r"9",
    "A": r"\A",
    "alpha": r"\alpha",
    "b": r"b",
    "beta": r"\beta",
    "C": r"\C",
    "cos": r"\cos",
    "d": r"d",
    "Delta": r"\Delta",
    "div": r"\div",
    "e": r"exp()",
    "exists": r"\exists",
    "f": r"f",
    "forall": r"\forall",
    "forward_slash": r"/",
    "G": r"\G",
    "gamma": r"\gamma",
    "geq": r"\geq",
    "gt": r">",
    "H": r"\H",
    "i": r"i",
    "in": r" \in",
    "infty": r"\infty",
    "int": r"\int",
    "j": r"j",
    "k": r"k",
    "l": r"l",
    "lambda": r"\lambda",
    "ldots": r"\ldots",
    "leq": r"\le",
    "lim": r"\lim",
    "log": r"\log",
    "lt": r"<",
    "M": r"\M",
    "mu": r"\mu",
    "N": r"\N",
    "neq": r"\neq",
    "o": r"\O",
    "p": r"p",
    "phi": r"\Phi",
    "pi": r"\Pi",
    "pm": r"\pm",
    "q": r"q",
    "R": r"\R",
    "rightarrow": r"\rightarrow",
    "S": r"\S",
    "sigma": r"\sigma",
    "sin": r"\sin",
    "sum": r"\sum",
    "T": r"\T",
    "tan": r"\tan",
    "theta": r"\theta",
    "times": r"\times",
    "u": r"u",
    "v": r"v",
    "w": r"w",
    "X": r"\X",
    "y": r"y",
    "z": r"z"
}

# Reading the kaggle [dataset](https://www.kaggle.com/datasets/xainano/handwrittenmathsymbols?resource=download)

Steps:
1. create a list of images and another list of labels for each image
2. store them in pickle files for easy retrieval when re-running the code 

In [309]:
def loadData(dataDir):
    imgs = []
    labels = []
    for key, value in dic.items():
        path = os.path.join(dataDir, key)
        for imgName in os.listdir(path):
            try:
                img = cv2.imread(os.path.join(path, imgName), cv2.COLOR_BGR2GRAY) 
                imgs.append(img)
                labels.append(value)
            except Exception as e:
                print(e)    
    return (imgs, labels)

The following cell is commented as it takes a long time (10min if image RGB, 1min otherwise) to create the pickle files

In [311]:
imgs, labels = loadData('mathSymbolsDataset/')
with open("x_symbols", 'wb') as f:
    pickle.dump(imgs, f)
with open("y_latex", 'wb') as f:
    pickle.dump(labels, f)

In [312]:
with open("x_symbols", 'rb') as f:
    imgs = pickle.load(f)
with open("y_latex", 'rb') as f:
    labels = pickle.load(f)

# converting text labels (latex) to numeric codes

In [344]:
LatexToNums = {k: v for v, k in enumerate(np.unique(labels))}
numsToLatex = {v: k for v, k in enumerate(np.unique(labels))}
